## PySAL Change Log Statistics: Table Generation

This notebook generates the summary statistics for use in the 6-month releases of PySAL, which is now a meta package. 

It assumes the subpackages have been git cloned in a directory below the location of this notebook. It also requires network connectivity for some of the reporting.

Run this notebook after `100-gitcount.ipynb`


In [1]:
from __future__ import print_function
import os
import json
import re
import sys
import pandas
import subprocess
from subprocess import check_output

#import yaml
from datetime import datetime, timedelta, time

from dateutil.parser import parse
import pytz

utc=pytz.UTC

try:
    from urllib import urlopen
except:
    from urllib.request import urlopen

from release_info import release_date, start_date, PYSALVER

since = datetime.combine(start_date, time(0,0))


In [2]:
release_date

datetime.date(2023, 1, 30)

In [3]:
CWD = os.path.abspath(os.path.curdir)

In [4]:
CWD

'/Users/serge/Documents/p/pysal/src/pysal/tools'

In [5]:
with open('frozen.txt', 'r') as package_list:
    packages = package_list.readlines()
    packages = dict([package.strip().split(">=") for package in packages])

In [6]:
packages

{'libpysal': '4.7.0',
 'access': '1.1.8',
 'esda': '2.4.1',
 'giddy': '2.3.3',
 'inequality': '1.0.0',
 'pointpats': '2.2.0',
 'segregation': '2.3.1',
 'spaghetti': '1.7.2',
 'mgwr': '2.1.2',
 'momepy': '0.5.4',
 'spglm': '1.0.8',
 'spint': '1.0.7',
 'spreg': '1.3.0',
 'spvcm': '0.3.0',
 'tobler': '0.8.2',
 'mapclassify': '2.5.0',
 'splot': '1.1.5.post1',
 'spopt': '0.5.0'}

import pysal
packages['pysal'] = pysal.__version__

In [7]:
import pickle

In [8]:
issues_closed = pickle.load(open("issues_closed.p", 'rb'))
pulls_closed = pickle.load(open('pulls_closed.p', 'rb'))

In [9]:
type(issues_closed)

dict

In [10]:
issues_closed.keys()

dict_keys(['libpysal', 'access', 'esda', 'giddy', 'inequality', 'pointpats', 'segregation', 'spaghetti', 'mgwr', 'momepy', 'spglm', 'spint', 'spreg', 'spvcm', 'tobler', 'mapclassify', 'splot', 'spopt', 'pysal'])

In [11]:
from release_info import get_pypi_info, get_github_info, clone_masters

In [12]:
#github_releases = get_github_info()

github_releases = pickle.load(open("releases.p", 'rb'))


In [13]:
from datetime import datetime

In [14]:
#pysal_date = datetime.strptime('2021-07-31T12:00:00Z', '%Y-%m-%dT%H:%M:%SZ')
pysal_date = release_date
#ISO8601 = "%Y-%m-%dT%H:%M:%SZ"


In [15]:
pysal_rel = {'version': f'v{PYSALVER}',
            'release_date': release_date}
github_releases['pysal'] = pysal_rel

In [16]:
github_releases

{'libpysal': {'version': 'v4.7.0',
  'url': 'https://api.github.com/repos/pysal/libpysal/tarball/v4.7.0',
  'release_date': datetime.datetime(2022, 11, 17, 22, 59, 5)},
 'access': {'version': 'v1.1.8',
  'url': 'https://api.github.com/repos/pysal/access/tarball/v1.1.8',
  'release_date': datetime.datetime(2022, 6, 4, 20, 15, 41)},
 'esda': {'version': 'v2.4.1',
  'url': 'https://api.github.com/repos/pysal/esda/tarball/v2.4.1',
  'release_date': datetime.datetime(2021, 7, 27, 12, 54, 27)},
 'giddy': {'version': 'v2.3.3',
  'url': 'https://api.github.com/repos/pysal/giddy/tarball/v2.3.3',
  'release_date': datetime.datetime(2020, 6, 10, 4, 59, 45)},
 'inequality': {'version': 'v1.0.0',
  'url': 'https://api.github.com/repos/pysal/inequality/tarball/v1.0.0',
  'release_date': datetime.datetime(2018, 10, 31, 22, 28, 18)},
 'pointpats': {'version': 'v2.2.0',
  'url': 'https://api.github.com/repos/pysal/pointpats/tarball/v2.2.0',
  'release_date': datetime.datetime(2020, 7, 27, 22, 17, 33)},

In [17]:
from datetime import datetime
datetime.fromtimestamp(0)
ISO8601 = "%Y-%m-%dT%H:%M:%SZ"


final_pulls = {}
final_issues = {}
for package in packages:
    filtered_issues = []
    filtered_pulls = []
    released = github_releases[package]['release_date']
    package_pulls = pulls_closed[package]
    package_issues = issues_closed[package]
    for issue in package_issues:
        #print(issue['number'], issue['title'], issue['closed_at'])
        closed = datetime.strptime(issue['closed_at'], ISO8601)
        if closed <= released and closed > since:
            filtered_issues.append(issue)
    final_issues[package] = filtered_issues
    for pull in package_pulls:
        #print(pull['number'], pull['title'], pull['closed_at'])
        closed = datetime.strptime(pull['closed_at'], ISO8601)
        if closed <= released and closed > since:
            filtered_pulls.append(pull)
    final_pulls[package] = filtered_pulls
    print(package, released, len(package_issues), len(filtered_issues), len(package_pulls),
         len(filtered_pulls))

libpysal 2022-11-17 22:59:05 29 10 0 0
access 2022-06-04 20:15:41 2 0 1 0
esda 2021-07-27 12:54:27 33 0 0 0
giddy 2020-06-10 04:59:45 9 0 0 0
inequality 2018-10-31 22:28:18 22 0 10 0
pointpats 2020-07-27 22:17:33 0 0 0 0
segregation 2022-06-18 19:58:17 8 0 0 0
spaghetti 2023-01-16 02:01:33 30 28 0 0
mgwr 2020-09-08 21:20:34 3 0 1 0
momepy 2022-09-20 14:49:58 66 41 0 0
spglm 2020-09-08 20:34:08 1 0 1 0
spint 2020-09-09 02:28:50 0 0 0 0
spreg 2022-10-28 19:59:45 11 5 5 4
spvcm 2020-02-02 19:42:39 0 0 0 0
tobler 2021-06-30 18:24:55 1 0 0 0
mapclassify 2023-01-14 15:50:46 44 42 15 15
splot 2022-04-13 21:13:27 11 0 5 0
spopt 2023-01-14 03:13:47 79 79 0 0


In [18]:
issue_details = final_issues
pull_details = final_pulls

In [19]:
packages

{'libpysal': '4.7.0',
 'access': '1.1.8',
 'esda': '2.4.1',
 'giddy': '2.3.3',
 'inequality': '1.0.0',
 'pointpats': '2.2.0',
 'segregation': '2.3.1',
 'spaghetti': '1.7.2',
 'mgwr': '2.1.2',
 'momepy': '0.5.4',
 'spglm': '1.0.8',
 'spint': '1.0.7',
 'spreg': '1.3.0',
 'spvcm': '0.3.0',
 'tobler': '0.8.2',
 'mapclassify': '2.5.0',
 'splot': '1.1.5.post1',
 'spopt': '0.5.0'}

In [20]:
github_releases['pysal']['release_date'] = release_date

In [21]:
released

datetime.datetime(2023, 1, 14, 3, 13, 47)

In [22]:
packages.keys()

dict_keys(['libpysal', 'access', 'esda', 'giddy', 'inequality', 'pointpats', 'segregation', 'spaghetti', 'mgwr', 'momepy', 'spglm', 'spint', 'spreg', 'spvcm', 'tobler', 'mapclassify', 'splot', 'spopt'])

In [23]:
spvcm = packages['spvcm']

In [24]:
## skip packages not released since last meta release

# handle meta
mrd = github_releases['pysal']['release_date']
github_releases['pysal']['release_date'] =  datetime.combine(mrd, time(0,0))


In [25]:
for package in github_releases:
    if github_releases[package]['release_date']>since:
        print("new: ",package)
    else:
        print('old:', package)

new:  libpysal
old: access
old: esda
old: giddy
old: inequality
old: pointpats
old: segregation
new:  spaghetti
old: mgwr
new:  momepy
old: spglm
old: spint
new:  spreg
old: spvcm
old: tobler
new:  mapclassify
old: splot
new:  spopt
new:  pysal


In [26]:
github_releases[package]['release_date']

datetime.datetime(2023, 1, 30, 0, 0)

In [27]:
since

datetime.datetime(2022, 7, 31, 0, 0)

In [28]:
github_releases

{'libpysal': {'version': 'v4.7.0',
  'url': 'https://api.github.com/repos/pysal/libpysal/tarball/v4.7.0',
  'release_date': datetime.datetime(2022, 11, 17, 22, 59, 5)},
 'access': {'version': 'v1.1.8',
  'url': 'https://api.github.com/repos/pysal/access/tarball/v1.1.8',
  'release_date': datetime.datetime(2022, 6, 4, 20, 15, 41)},
 'esda': {'version': 'v2.4.1',
  'url': 'https://api.github.com/repos/pysal/esda/tarball/v2.4.1',
  'release_date': datetime.datetime(2021, 7, 27, 12, 54, 27)},
 'giddy': {'version': 'v2.3.3',
  'url': 'https://api.github.com/repos/pysal/giddy/tarball/v2.3.3',
  'release_date': datetime.datetime(2020, 6, 10, 4, 59, 45)},
 'inequality': {'version': 'v1.0.0',
  'url': 'https://api.github.com/repos/pysal/inequality/tarball/v1.0.0',
  'release_date': datetime.datetime(2018, 10, 31, 22, 28, 18)},
 'pointpats': {'version': 'v2.2.0',
  'url': 'https://api.github.com/repos/pysal/pointpats/tarball/v2.2.0',
  'release_date': datetime.datetime(2020, 7, 27, 22, 17, 33)},

In [29]:
since_date = '--since="{start}"'.format(start=start_date.strftime("%Y-%m-%d"))
since_date
                                        

'--since="2022-07-31"'

In [30]:
# commits
cmd = ['git', 'log', '--oneline', since_date]

activity = {}
total_commits = 0
tag_dates = {}
ncommits_total = 0
for subpackage in packages:
    released = github_releases[subpackage]['release_date']
    tag_date = released.strftime("%Y-%m-%d")
    tag_dates[subpackage] = tag_date
    print(tag_date)
    #tag_date = tag_dates[subpackage]
    ncommits = 0
    if released > since:
        os.chdir(CWD)
        os.chdir('tmp/{subpackage}'.format(subpackage=subpackage))
        cmd_until = cmd + ['--until="{tag_date}"'.format(tag_date=tag_date)]
        ncommits = len(check_output(cmd_until).splitlines())
        ncommits_total = len(check_output(cmd).splitlines())
    print(subpackage, ncommits_total, ncommits, tag_date)
    total_commits += ncommits
    activity[subpackage] = ncommits

2022-11-17
libpysal 62 34 2022-11-17
2022-06-04
access 62 0 2022-06-04
2021-07-27
esda 62 0 2021-07-27
2020-06-10
giddy 62 0 2020-06-10
2018-10-31
inequality 62 0 2018-10-31
2020-07-27
pointpats 62 0 2020-07-27
2022-06-18
segregation 62 0 2022-06-18
2023-01-16
spaghetti 68 68 2023-01-16
2020-09-08
mgwr 68 0 2020-09-08
2022-09-20
momepy 37 26 2022-09-20
2020-09-08
spglm 37 0 2020-09-08
2020-09-09
spint 37 0 2020-09-09
2022-10-28
spreg 11 9 2022-10-28
2020-02-02
spvcm 11 0 2020-02-02
2021-06-30
tobler 11 0 2021-06-30
2023-01-14
mapclassify 60 60 2023-01-14
2022-04-13
splot 60 0 2022-04-13
2023-01-14
spopt 241 241 2023-01-14


In [31]:
activity

{'libpysal': 34,
 'access': 0,
 'esda': 0,
 'giddy': 0,
 'inequality': 0,
 'pointpats': 0,
 'segregation': 0,
 'spaghetti': 68,
 'mgwr': 0,
 'momepy': 26,
 'spglm': 0,
 'spint': 0,
 'spreg': 9,
 'spvcm': 0,
 'tobler': 0,
 'mapclassify': 60,
 'splot': 0,
 'spopt': 241}

In [32]:
subpackage

'spopt'

In [33]:
CWD

'/Users/serge/Documents/p/pysal/src/pysal/tools'

In [34]:
# commits
cmd = ['git', 'log', '--oneline', since_date]

activity = {}
total_commits = 0
for subpackage in packages:
    ncommits = 0
    tag_date = tag_dates[subpackage]
    released = github_releases[subpackage]['release_date']
    if released > since:
        os.chdir(CWD)
        os.chdir('tmp/{subpackage}'.format(subpackage=subpackage))
        cmd_until = cmd + ['--until="{tag_date}"'.format(tag_date=tag_date)]
        ncommits = len(check_output(cmd_until).splitlines())
        print(ncommits)
        ncommits_total = len(check_output(cmd).splitlines())
        print(subpackage, ncommits_total, ncommits, tag_date)
    total_commits += ncommits
    activity[subpackage] = ncommits

34
libpysal 62 34 2022-11-17
68
spaghetti 68 68 2023-01-16
26
momepy 37 26 2022-09-20
9
spreg 11 9 2022-10-28
60
mapclassify 60 60 2023-01-14
241
spopt 241 241 2023-01-14


In [35]:
identities = {'Levi John Wolf': ('ljwolf', 'Levi John Wolf'),
              'Serge Rey': ('Serge Rey', 'Sergio Rey', 'sjsrey', 'serge'),
              'Wei Kang': ('Wei Kang', 'weikang9009'),
              'Dani Arribas-Bel': ('Dani Arribas-Bel', 'darribas'),
              'Antti Härkönen': ( 'antth', 'Antti Härkönen', 'Antti Härkönen', 'Antth'  ),
              'Juan C Duque': ('Juan C Duque', "Juan Duque"),
              'Renan Xavier Cortes': ('Renan Xavier Cortes', 'renanxcortes', 'Renan Xavier Cortes'   ),
              'Taylor Oshan': ('Tayloroshan', 'Taylor Oshan', 'TaylorOshan'),
              'Tom Gertin': ('@Tomgertin', 'Tom Gertin', '@tomgertin')
}

def regularize_identity(string):
    string = string.decode()
    for name, aliases in identities.items():
        for alias in aliases:
            if alias in string:
                string = string.replace(alias, name)
    if len(string.split(' '))>1:
        string = string.title()
    return string.lstrip('* ')

In [36]:
author_cmd = ['git', 'log', '--format=* %aN', since_date]

In [37]:
author_cmd.append('blank')

In [38]:
author_cmd

['git', 'log', '--format=* %aN', '--since="2022-07-31"', 'blank']

In [39]:
from collections import Counter

In [40]:
tag_dates

{'libpysal': '2022-11-17',
 'access': '2022-06-04',
 'esda': '2021-07-27',
 'giddy': '2020-06-10',
 'inequality': '2018-10-31',
 'pointpats': '2020-07-27',
 'segregation': '2022-06-18',
 'spaghetti': '2023-01-16',
 'mgwr': '2020-09-08',
 'momepy': '2022-09-20',
 'spglm': '2020-09-08',
 'spint': '2020-09-09',
 'spreg': '2022-10-28',
 'spvcm': '2020-02-02',
 'tobler': '2021-06-30',
 'mapclassify': '2023-01-14',
 'splot': '2022-04-13',
 'spopt': '2023-01-14'}

In [41]:
authors_global = set()
authors = {}
global_counter = Counter()
counters = dict()
cmd = ['git', 'log', '--oneline', since_date]
total_commits = 0
activity = {}
for subpackage in packages:
    ncommits = 0
    released = github_releases[subpackage]['release_date']
    if released > since:
        os.chdir(CWD)
        os.chdir('tmp/{subpackage}'.format(subpackage=subpackage))
        ncommits = len(check_output(cmd).splitlines())
        print(cmd)
        tag_date = tag_dates[subpackage]
        tag_date = (datetime.strptime(tag_date, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d')
        author_cmd[-1] = '--until="{tag_date}"'.format(tag_date=tag_date)
        #cmd_until = cmd + ['--until="{tag_date}"'.format(tag_date=tag_date)]
        print(subpackage, author_cmd)


        all_authors = check_output(author_cmd).splitlines()
        counter = Counter([regularize_identity(author) for author in all_authors])
        global_counter += counter
        counters.update({subpackage: counter})
        unique_authors = sorted(set(all_authors))
        authors[subpackage] =  unique_authors
        authors_global.update(unique_authors)
    total_commits += ncommits
    activity[subpackage] = ncommits

['git', 'log', '--oneline', '--since="2022-07-31"']
libpysal ['git', 'log', '--format=* %aN', '--since="2022-07-31"', '--until="2022-11-18"']
['git', 'log', '--oneline', '--since="2022-07-31"']
spaghetti ['git', 'log', '--format=* %aN', '--since="2022-07-31"', '--until="2023-01-17"']
['git', 'log', '--oneline', '--since="2022-07-31"']
momepy ['git', 'log', '--format=* %aN', '--since="2022-07-31"', '--until="2022-09-21"']
['git', 'log', '--oneline', '--since="2022-07-31"']
spreg ['git', 'log', '--format=* %aN', '--since="2022-07-31"', '--until="2022-10-29"']
['git', 'log', '--oneline', '--since="2022-07-31"']
mapclassify ['git', 'log', '--format=* %aN', '--since="2022-07-31"', '--until="2023-01-15"']
['git', 'log', '--oneline', '--since="2022-07-31"']
spopt ['git', 'log', '--format=* %aN', '--since="2022-07-31"', '--until="2023-01-15"']


In [42]:
author_cmd

['git',
 'log',
 '--format=* %aN',
 '--since="2022-07-31"',
 '--until="2023-01-15"']

In [43]:
subpackage

'spopt'

In [44]:
authors_global

{b'* Andres Morfin Veytia',
 b'* Erin Olson',
 b'* Germano Barcelos',
 b'* Greg Maya',
 b'* James Gaboardi',
 b'* Levi John Wolf',
 b'* Martin Fleischmann',
 b'* Nick Bearman',
 b'* Pedro Amaral',
 b'* Serge Rey',
 b'* Sergio Rey',
 b'* Thi',
 b'* dependabot[bot]',
 b'* github-actions[bot]',
 b'* ljwolf',
 b'* pedrovma',
 b'* pre-commit-ci[bot]'}

In [45]:
activity

{'libpysal': 62,
 'access': 0,
 'esda': 0,
 'giddy': 0,
 'inequality': 0,
 'pointpats': 0,
 'segregation': 0,
 'spaghetti': 68,
 'mgwr': 0,
 'momepy': 37,
 'spglm': 0,
 'spint': 0,
 'spreg': 11,
 'spvcm': 0,
 'tobler': 0,
 'mapclassify': 60,
 'splot': 0,
 'spopt': 241}

In [46]:
def get_tag(title, level="##", as_string=True):
    words = title.split()
    tag = "-".join([word.lower() for word in words])
    heading = level+" "+title
    line = "\n\n<a name=\"{}\"></a>".format(tag)
    lines = [line]
    lines.append(heading)
    if as_string:
        return "\n".join(lines)
    else:
        return lines

In [47]:
subs = issue_details.keys()
table = []
txt = []
lines = get_tag("Changes by Package", as_string=False)

for sub in subs:
    total= issue_details[sub]
    pr = pull_details[sub]
    
    row = [sub, activity[sub], len(total), len(pr)]
    table.append(row)
    #line = "\n<a name=\"{sub}\"></a>".format(sub=sub)
    #lines.append(line)
    #line = "### {sub}".format(sub=sub)
    #lines.append(line)
    lines.extend(get_tag(sub.lower(), "###", as_string=False))
    for issue in total:
        url = issue['html_url']
        title = issue['title']
        number = issue['number']
        line = "* [#{number}:]({url}) {title} ".format(title=title,
                                                     number=number,
                                                     url=url)
        lines.append(line)



In [48]:
sub

'spopt'

In [49]:
os.chdir(CWD)

import pandas

In [50]:
df = pandas.DataFrame(table, columns=['package', 'commits', 'total issues', 'pulls'])

In [51]:
df.head()

package  commits  total issues  pulls
0    libpysal       62            10      0
1      access        0             0      0
2        esda        0             0      0
3       giddy        0             0      0
4  inequality        0             0      0

In [52]:
df.shape

(18, 4)

In [53]:
type(counters)

dict

In [54]:
df.sort_values(['commits','pulls'], ascending=False)\
  .to_html('./commit_table.html', index=None)

In [55]:
df.sum()

package         libpysalaccessesdagiddyinequalitypointpatssegr...
commits                                                       479
total issues                                                  205
pulls                                                          19
dtype: object

In [56]:
contributor_table = pandas.DataFrame.from_dict(counters).fillna(0).astype(int).T

In [57]:
contributor_table.to_html('./contributor_table.html')

In [58]:
totals = contributor_table.sum(axis=0).T
totals.sort_index().to_frame('commits')

commits
Andres Morfin Veytia        1
Dependabot[Bot]             3
Erin Olson                137
Germano Barcelos            9
Github-Actions[Bot]         7
Greg Maya                   2
James Gaboardi            222
Levi John Wolf              5
Martin Fleischmann         20
Nick Bearman                1
Pedro Amaral                3
Pedrovma                    2
Pre-Commit-Ci[Bot]         15
Serge Rey                  11
Thi                         1

In [59]:
totals = contributor_table.sum(axis=0).T
totals.sort_index().to_frame('commits').to_html('./commits_by_person.html')

In [60]:
totals

James Gaboardi          222
Serge Rey                11
Martin Fleischmann       20
Levi John Wolf            5
Github-Actions[Bot]       7
Pre-Commit-Ci[Bot]       15
Dependabot[Bot]           3
Greg Maya                 2
Thi                       1
Andres Morfin Veytia      1
Pedro Amaral              3
Pedrovma                  2
Germano Barcelos          9
Nick Bearman              1
Erin Olson              137
dtype: int64

In [61]:
n_commits = df.commits.sum()
n_issues = df['total issues'].sum()
n_pulls = df.pulls.sum()

In [62]:
n_commits

479

In [63]:
#Overall, there were 719 commits that closed 240 issues, together with 105 pull requests across 12 packages since our last release on 2017-11-03.
#('{0} Here is a really long '
#           'sentence with {1}').format(3, 5))
line = ('Overall, there were {n_commits} commits that closed {n_issues} issues,'  
    ' together with {n_pulls} pull requests since our last release' 
        ' on {since_date}.\n'.format(n_commits=n_commits, n_issues=n_issues,
        n_pulls=n_pulls, since_date = start_date))

In [64]:
line

'Overall, there were 479 commits that closed 205 issues, together with 19 pull requests since our last release on 2022-07-31.\n'

## append html files to end of changes.md with tags for toc

In [65]:
with open('changes.md', 'w') as fout:
    fout.write(line)
    fout.write("\n".join(lines))
    fout.write(get_tag("Contributors"))
    fout.write("\n\nMany thanks to all of the following individuals who contributed to this release:\n\n")
    
    
    
    totals = contributor_table.sum(axis=0).T
    contributors = totals.index.values
    contributors.sort()
    contributors = contributors.tolist() 
    contributors = [ f'\n - {contributor}' for contributor in contributors]
    fout.write("".join(contributors))
    
